In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import json
import datetime

In [2]:
df = pd.read_csv("dataset_cleaned.csv")

In [3]:
df.head()

,time,latitude,longitude,depth,mag,nst,gap,dmin,rms,updated,...,depthError,magError,magNst,status,year,month,day,h:m:s,state,city
0,2021-10-18 22:00:00+00:00,40.467999,-124.871002,25.90,2.59,8.0,325.0,0.47120,0.19,2021-10-19T00:36:12.127Z,...,11.58,0.160,3.0,automatic,2021,10,18,22:00:00,California,Ferndale
1,2021-10-18 20:40:39+00:00,37.287498,-120.031502,25.39,2.73,8.0,72.0,0.11900,0.08,2021-10-19T00:29:39.269Z,...,0.99,0.200,11.0,automatic,2021,10,18,20:40:39,California,Mariposa
2,2021-10-18 10:14:20+00:00,36.809166,-121.546166,0.70,2.81,40.0,154.0,0.28730,0.19,2021-10-18T17:05:27.120Z,...,2.25,0.110,41.0,automatic,2021,10,18,10:14:20,California,San Juan Bautista
3,2021-10-18 09:45:06+00:00,36.185833,-120.309167,10.36,3.44,45.0,107.0,0.06493,0.08,2021-10-19T03:24:17.760Z,...,0.28,0.192,87.0,reviewed,2021,10,18,9:45:06,California,Coalinga
4,2021-10-18 09:42:15+00:00,36.192001,-120.314331,9.70,2.67,42.0,106.0,0.10650,0.14,2021-10-18T11:22:59.040Z,...,0.55,0.170,49.0,automatic,2021,10,18,9:42:15,California,Coalinga


In [56]:
time_ascending = df.sort_values(by = "time", ascending = True)
time_ascending["year_month"] = pd.to_datetime(time_ascending[['year', 'month', 'day']]).dt.strftime("%Y-%m")
#time_ascending["year_month"] = str(time_ascending["year_month"])
time_ascending.head()

,time,latitude,longitude,depth,mag,nst,gap,dmin,rms,updated,...,magError,magNst,status,year,month,day,h:m:s,state,city,year_month
12616,2018-10-18 02:11:00+00:00,40.372500,-124.602833,23.710,2.56,30.000000,241.0,0.227200,0.17,2018-10-19T01:57:04.730Z,...,0.218000,30.000000,reviewed,2018,10,18,2:11:00,California,Ferndale,2018-10
12615,2018-10-18 02:11:13+00:00,37.079200,-97.820200,7.250,2.50,42.158718,64.0,0.056000,0.52,2018-12-21T22:27:04.040Z,...,0.117000,10.000000,reviewed,2018,10,18,2:11:13,Kansas,Caldwell,2018-10
12614,2018-10-18 04:05:34+00:00,34.044100,-97.408200,6.684,2.60,42.158718,53.0,0.226275,0.58,2018-12-21T22:27:04.040Z,...,0.159465,42.746715,reviewed,2018,10,18,4:05:34,Oklahoma,Wilson,2018-10
12613,2018-10-18 05:50:48+00:00,33.311333,-116.345000,14.030,2.58,112.000000,26.0,0.024070,0.20,2018-10-23T16:30:00.260Z,...,0.178000,124.000000,reviewed,2018,10,18,5:50:48,California,Borrego Springs,2018-10
12612,2018-10-18 06:28:14+00:00,37.521333,-118.102667,2.070,2.61,39.000000,154.0,0.123500,0.08,2018-10-19T04:28:03.584Z,...,0.186000,34.000000,reviewed,2018,10,18,6:28:14,California,Bishop,2018-10


In [71]:
time_ascending[time_ascending["type"] == "experimental explosion"]

,time,latitude,longitude,depth,mag,nst,gap,dmin,rms,updated,...,magError,magNst,status,year,month,day,h:m:s,state,city,year_month
1188,2021-06-18 19:48:39+00:00,29.7433,-79.3493,0.0,3.9,42.158718,108.0,2.445,0.58,2021-10-12T02:09:54.857Z,...,0.059,38.0,reviewed,2021,6,18,19:48:39,Florida,Ponce Inlet,2021-06
795,2021-07-16 17:48:33+00:00,30.0852,-79.3251,0.0,3.9,42.158718,110.0,2.415,0.75,2021-09-25T21:07:51.040Z,...,0.055,44.0,reviewed,2021,7,16,17:48:33,Florida,Ormond-by-the-Sea,2021-07
558,2021-08-08 19:55:18+00:00,30.1465,-79.6066,0.0,3.9,42.158718,109.0,2.164,0.82,2021-09-04T19:23:26.228Z,...,0.055,44.0,reviewed,2021,8,8,19:55:18,Florida,Flagler Beach,2021-08


In [100]:
fig = px.scatter_mapbox(time_ascending,
              lat="latitude",
              lon="longitude",
              hover_name="type",
              color="mag",
              opacity=0.5,
              color_continuous_scale="sunsetdark",
              animation_frame="year_month",
              mapbox_style='open-street-map',  
              title = "Earthquake Occurrences Over Time",
              zoom=2.5)
#fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
#fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 5
fig.show()

In [101]:
fig.write_html("earthquakes_animation.html")

In [102]:
fig = px.scatter_mapbox(time_ascending[time_ascending["type"] != "earthquake"],
              lat="latitude",
              lon="longitude",
              hover_name="type",
              color="mag",
              opacity=0.5,
              color_continuous_scale="sunsetdark",
              mapbox_style='open-street-map',  
              title = "Earthquake Occurrences Over Time",
              zoom=2.5)
#fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
#fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 5
fig.show()